# 📘 Topology Capsules v1.0

**Overview**
This notebook introduces *Topology Capsules*, a graph-based framework for learning and inference without deep training loops. Instead of relying on backpropagation, the method builds *topological capsules* — compact spectral structures extracted from graphs of data — and uses them for semi-supervised label propagation and transfer.

The core idea: rather than treating similarity and dissimilarity symmetrically, *Topology Capsules* explicitly model both **positive affinities** (who is close) and **negative repulsions** (who is far), and reconcile them through signed graph Laplacians. This gives a capsule-like latent space that can be probed with very few labels.

---

**Key Features in v1.0**

* **Multi-view construction**: Each dataset is rotated, jittered, and embedded in multiple metric views. Only neighbors consistent across views are kept, enforcing *mutual consensus*.
* **Dual neighborhoods**:

  * **KNN(+) positive links** capture local similarity.
  * **KFN(−) negative links** capture structured dissimilarity by finding farthest-neighbors in embedding space.
* **Adaptive affinities**:

  * *Positive edges*: Zelnik–Manor scaling (local bandwidths).
  * *Negative edges*: Robust z-score normalization with smooth sigmoid repulsion.
* **Capsule freezing**: The capsule basis (eigenvectors of the positive Laplacian) is frozen; labels only probe it via signed label propagation.
* **Seed-only auto-tuning**: Hyperparameters (repulsion weight λ, ridge term) are tuned using only labeled seeds, avoiding leakage from the unlabeled pool.
* **Internal transfer**: Capsule posteriors can transfer across held-out slices via lightweight kNN pulls in spectral space.

---

**Benchmarks Included**

1. **Fashion-MNIST (pixels → PCA → Topology Capsule)**
   Tests the core CNTC v2 pipeline with consensus neighbors, positive/negative edges, and signed label propagation.
2. **CIFAR-10 (HOG + color statistics)**
   Builds capsules using fast CPU-friendly features (no CNNs), showing capsule inference with a few labels per class.
3. **Cat vs Dog Capsule (multi-view: silhouette, texture, structure, diffusion)**
   Demonstrates ensemble capsules with per-view calibration, perturbation-based stability weighting, and geometric-mean fusion.

---

**Why It Matters**
Topology Capsules bridge geometric learning and graph signal processing:

* They sidestep costly neural training.
* They expose both attractive and repulsive structure in the data.
* They work in low-label settings, making them ideal for semi-supervised problems.
* They are interpretable: capsules are frozen spectral bases with clear diagnostics (eigenvalues, connectivity, stability).

---

⚡ *In short: Topology Capsules v1.0 explores how to capture the “shape” of data through graphs and use that shape directly for learning — robust, interpretable, and lightweight.*

In [ ]:
# ===============================================================
# CNTC v2 — Stronger Pd via mutual consensus + true local scaling
# - Multi-view (random orthonormal rotations + jitter)
# - Mutual-consensus KNN(+), robust KFN(−)
# - Zelnik–Manor positive affinities; robust z-score repulsion
# - Sparse eigsh; capsule frozen; labels only probe it
# - Seed-only auto-tune of (lam_neg, ridge) for signed LP
# ===============================================================

!pip -q install torchvision scikit-learn

import numpy as np, random, math, time
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score, log_loss
from scipy.sparse import coo_matrix, csr_matrix, diags, eye
from scipy.sparse.linalg import eigsh, spsolve
from scipy.sparse.csgraph import connected_components
from torchvision import datasets, transforms

# -------------------------------
# Config
# -------------------------------
SEED = 7
rng = np.random.default_rng(SEED)
random.seed(SEED); np.random.seed(SEED)

N_POINTS   = 6000
PCA_DIM    = 64
VIEWS      = 4           # more diverse than 3
KNN_K      = 16
KFN_K      = 8
KEEP_KPOS  = 12
KEEP_KNEG  = 6
CONS_THR   = 2           # ≥ this many views
RHO_ROT    = 0.20        # rotation mix strength
JITTER_STD = 0.01

EIGS       = 64
LABELED_PER_CLASS = 40
TEST_SPLIT = 0.85

# grids for signed LP (seed-only tuning)
LAM_GRID   = [0.0, 0.05, 0.1, 0.15, 0.2]
RIDGE_GRID = [1e-5, 5e-5, 1e-4, 5e-4]

t0 = time.time()

# -------------------------------
# 1) Data → PCA coords (distances only)
# -------------------------------
tf = transforms.Compose([transforms.ToTensor()])
ds = datasets.FashionMNIST(root='./data', train=True, download=True, transform=tf)

idx = np.arange(len(ds)); rng.shuffle(idx); idx = idx[:N_POINTS]
X = np.stack([ds[i][0].numpy().reshape(-1) for i in idx], 0).astype(np.float32)/255.0
y = np.array([ds[i][1] for i in idx], dtype=np.int64)
del ds

Xp = StandardScaler().fit_transform(X); del X
Z0 = PCA(n_components=PCA_DIM, random_state=SEED).fit_transform(Xp).astype(np.float32); del Xp

def normalize_rows(A): return A / (np.linalg.norm(A, axis=1, keepdims=True)+1e-9)

# Random orthonormal rotation with mix
def rotate_view(Z, rho=0.2, jitter=0.01):
    m = Z.shape[1]
    G = rng.standard_normal((m, m)).astype(np.float32)
    Q,_ = np.linalg.qr(G)                          # orthonormal
    M = (1.0-rho)*np.eye(m, dtype=np.float32) + rho*Q
    Zv = Z @ M
    return Zv + rng.normal(0.0, jitter, size=Z.shape).astype(np.float32)

# -------------------------------
# 2) Multi-view neighbors: KNN(+), KFN(−) via “nearest to −z”
# -------------------------------
def view_neighbors(Z, k_pos, k_neg):
    Zc = normalize_rows(Z)
    nn = NearestNeighbors(n_neighbors=k_pos+1, metric='cosine').fit(Zc)
    d_pos, i_pos = nn.kneighbors(Zc)               # includes self
    i_pos = i_pos[:,1:]; d_pos = d_pos[:,1:]
    # farthest: nearest to -Zc
    d_neg, i_neg = nn.kneighbors(-Zc, n_neighbors=k_neg)
    # note: no self-match here since query = -z
    return (i_pos, d_pos), (i_neg, d_neg)

views_pos, views_neg = [], []
for _ in range(VIEWS):
    (ip,dp),(in_,dn_) = view_neighbors(rotate_view(Z0, RHO_ROT, JITTER_STD), KNN_K, KFN_K)
    views_pos.append((ip.astype(np.int32), dp.astype(np.float32)))
    views_neg.append((in_.astype(np.int32), dn_.astype(np.float32)))

N = len(y)

# -------------------------------
# 3) Consensus merge per node with mutuality
# -------------------------------
def merge_consensus(views, keep_k, mode="pos"):
    # Collect edge -> (count, sum_d)
    idx_out = np.zeros((N, keep_k), dtype=np.int32)
    d_out   = np.zeros((N, keep_k), dtype=np.float32)
    c_out   = np.zeros((N, keep_k), dtype=np.int8)
    bags = []
    for i in range(N):
        acc = {}
        for (nei,dist) in views:
            for j, d in zip(nei[i], dist[i]):
                if j==i: continue
                if j not in acc: acc[j] = [0, 0.0]
                acc[j][0] += 1
                acc[j][1] += float(d)
        bags.append(acc)

    # mutual: keep only i<->j that appear on both sides (after merge)
    for i in range(N):
        items = []
        for j,(cnt,sumd) in bags[i].items():
            if j>=N: continue
            if i in bags[j]:      # reciprocal at merged level
                cnt2,_ = bags[j][i]
                # require both sides to meet consensus
                if cnt>=CONS_THR and cnt2>=CONS_THR:
                    mean_d = sumd/cnt
                    items.append((j, cnt, mean_d))
        if len(items)<keep_k:
            # backfill with highest-count neighbors even if < CONS_THR
            for j,(cnt,sumd) in bags[i].items():
                if len(items)>=keep_k: break
                if all(j!=t[0] for t in items):
                    items.append((j, cnt, sumd/max(1,cnt)))
        if mode=="pos":
            items.sort(key=lambda t: (t[1], -t[2]), reverse=True)     # more views, then smaller d
        else:
            items.sort(key=lambda t: (t[1], t[2]),  reverse=True)     # more views, then larger "far" metric (we’ll convert)
        items = items[:keep_k]
        if len(items)>0:
            idx_out[i,:len(items)] = [t[0] for t in items]
            d_out[i,:len(items)]   = [t[2] for t in items]
            c_out[i,:len(items)]   = [t[1] for t in items]
    return idx_out, d_out, c_out

# POS: cosine distance d_pos in [0,2]
i_pos, d_pos, c_pos = merge_consensus(views_pos, KEEP_KPOS, mode="pos")
# NEG: we currently stored d_neg = cosine distance between (-z, j) in [0,2]
# Convert to "far distance" w.r.t original z: d_far = 2 - d_neg (= 1 - cos(z,j) + 1)
# Larger d_far == more dissimilar (max 2 when cos=-1)
i_neg_raw, d_neg_raw, c_neg = merge_consensus(views_neg, KEEP_KNEG, mode="neg")
d_far = 2.0 - d_neg_raw

# -------------------------------
# 4) Affinities
# 4a) Positive: Zelnik–Manor w_ij = exp(-d_ij^2 / (σ_i σ_j)) * (cnt/V)^γ
#     σ_i = kth neighbor distance (pos) — robust local scale
# 4b) Negative: robust z-score on d_far, r_ij = sigmoid( (z_i + z_j)/2 ) * (cnt/V)^γ
# -------------------------------
gamma = 1.0
V = float(VIEWS)

# POS local scales
sigma_pos = np.sort(d_pos, axis=1)[:, min(KEEP_KPOS-1, max(1, KEEP_KPOS//2))]
sigma_pos = sigma_pos.reshape(-1,1) + 1e-6

# POS weights (edgewise σ_i σ_j with sparse assembly)
rows_p = np.repeat(np.arange(N)[:,None], KEEP_KPOS, 1).ravel()
cols_p = i_pos.ravel()
dij2   = (d_pos.ravel()**2).astype(np.float32)
sig_i  = sigma_pos[rows_p,0]
sig_j  = sigma_pos[cols_p,0]
w_pos  = np.exp(-dij2 / ( (sig_i*sig_j) + 1e-9 )).astype(np.float32)
w_pos *= ((c_pos.ravel().astype(np.float32)/V)**gamma)

Wpos = coo_matrix((w_pos, (rows_p, cols_p)), shape=(N,N)).tocsr()
Wpos = Wpos.maximum(Wpos.T); Wpos.setdiag(0.0); Wpos.eliminate_zeros()

# NEG robust z-score
# robust center/scale from far distances (already have d_far, mu_neg, mad_neg)
SCALE_FLOOR = 1e-2      # floors tiny MAD to avoid exploding z
Z_CLIP      = 8.0       # clip z for numerical stability
REP_TEMP    = 0.75      # soften repulsion; >1 = softer, <1 = sharper

scale_neg = 1.4826*mad_neg + 1e-9
scale_neg = np.maximum(scale_neg, SCALE_FLOOR).astype(np.float32)

rows_n = np.repeat(np.arange(N)[:,None], KEEP_KNEG, 1).ravel()
cols_n = i_neg_raw.ravel()
df     = d_far.ravel().astype(np.float32)

# z-score on far distances for both endpoints, averaged
zi = (df - mu_neg[rows_n,0]) / scale_neg[rows_n,0]
zj = (df - mu_neg[cols_n,0]) / scale_neg[cols_n,0]
z  = 0.5*(zi + zj)
# clip and temperature-scale
z  = np.clip(z/REP_TEMP, -Z_CLIP, Z_CLIP)

# numerically stable sigmoid (tanh-based)
rep = (0.5*(1.0 + np.tanh(0.5*z))).astype(np.float32)

# consensus boost (already computed as c_neg)
rep *= ((c_neg.ravel().astype(np.float32) / float(VIEWS)) ** gamma)
rep = np.maximum(rep, 1e-6).astype(np.float32)

Wneg = coo_matrix((rep, (rows_n, cols_n)), shape=(N, N)).tocsr()
Wneg = Wneg.maximum(Wneg.T); Wneg.setdiag(0.0); Wneg.eliminate_zeros()

# Diagnostics
n_comp, labels_comp = connected_components(Wpos>0.0, directed=False)
deg_pos = np.array(Wpos.sum(axis=1)).ravel()
deg_neg = np.array(Wneg.sum(axis=1)).ravel()
print(f"Graphs built in {time.time()-t0:.1f}s | pos edges:{Wpos.nnz//2:,} | neg edges:{Wneg.nnz//2:,} | components:{n_comp} | deg_pos mean:{deg_pos.mean():.2f}")

# -------------------------------
# 5) Laplacians & spectral basis
# -------------------------------
def norm_laplacian(W):
    d = np.array(W.sum(axis=1)).ravel()+1e-8
    Dmh = diags((1.0/np.sqrt(d)).astype(np.float32))
    S   = Dmh @ W @ Dmh
    return (eye(W.shape[0], dtype=np.float32, format='csr') - S).tocsr()

Lpos = norm_laplacian(Wpos)
Lneg = norm_laplacian(Wneg)
I = eye(N, dtype=np.float32, format='csr')
Lneg_minus = (2.0*I - Lneg).tocsr()  # I + Sneg

e_vals, e_vecs = eigsh(Lpos, k=min(EIGS, N-2), which='SM', tol=1e-3)
idxs = np.argsort(e_vals); e_vals = e_vals[idxs].astype(np.float32); e_vecs = e_vecs[:,idxs].astype(np.float32)

TopologyCapsule = {
    "N": N,
    "Wpos": Wpos, "Wneg": Wneg,
    "Lpos": Lpos, "Lneg_minus": Lneg_minus,
    "evecs_pos": e_vecs, "evals_pos": e_vals,
    "KNN_K": KEEP_KPOS, "KFN_K": KEEP_KNEG
}
del Z0

# -------------------------------
# 6) Labeled/unlabeled split (few labels per class)
# -------------------------------
def few_labels_per_class(y, n_per_class):
    idx = np.arange(len(y)); rng.shuffle(idx)
    cls = [[] for _ in range(int(y.max()+1))]
    for i in idx:
        c = y[i]
        if len(cls[c])<n_per_class: cls[c].append(i)
    L = np.array([i for c in cls for i in c], dtype=int)
    U = np.setdiff1d(np.arange(len(y)), L, assume_unique=False)
    return L,U

idxL, idxU = few_labels_per_class(y, LABELED_PER_CLASS)

# -------------------------------
# 7) LP solvers (sparse)
# -------------------------------
def lp_solve(Lmat, idxL, y, ridge=1e-4):
    N = Lmat.shape[0]; C = int(y.max()+1)
    A = (Lmat + ridge*I).tocsr()
    YL = np.eye(C, dtype=np.float32)[y[idxL]]
    idxU = np.setdiff1d(np.arange(N), idxL, assume_unique=False)
    A_uu = A[idxU][:,idxU]; A_ul = A[idxU][:,idxL]
    Fu = spsolve(A_uu, -A_ul @ YL)
    F  = np.zeros((N,C), dtype=np.float32)
    F[idxL] = YL; F[idxU] = Fu
    P  = np.clip(F, 1e-9, None); P /= (P.sum(1, keepdims=True)+1e-9)
    return P

# KNN-only Pd (baseline)
Pd = lp_solve(Lpos, idxL, y, ridge=1e-4)
acc_pd = accuracy_score(y[idxU], Pd[idxU].argmax(1))

# Auto-tune signed Pd on seeds (log-likelihood on idxL only)
best = (-1e9, None, None, None)
for lam in LAM_GRID:
    A = (Lpos + lam*Lneg_minus)
    for rg in RIDGE_GRID:
        P_try = lp_solve(A, idxL, y, ridge=rg)
        ll = float(np.mean(np.log(np.clip(P_try[idxL, y[idxL]], 1e-12, 1.0))))
        if ll > best[0]:
            best = (ll, lam, rg, P_try)
ll_seed, lam_star, ridge_star, Psigned = best
acc_ps  = accuracy_score(y[idxU], Psigned[idxU].argmax(1))

print(f"[CNTC v2] Unlabeled acc | Pd: {acc_pd:.3f} | Pd(signed λ={lam_star}, ridge={ridge_star}): {acc_ps:.3f} | seed-LL={ll_seed:.4f}")

# -------------------------------
# 8) Internal transfer (kNN pull on spectral coords, using signed best)
# -------------------------------
Upos = TopologyCapsule["evecs_pos"]
split = int(TEST_SPLIT*N)
tr = np.arange(N)[:split]; te = np.arange(N)[split:]

def knn_pull(train_X, train_post, test_X, k=12):
    nn = NearestNeighbors(n_neighbors=k, metric='cosine').fit(train_X)
    d,i = nn.kneighbors(test_X)
    w = 1.0-d; w /= (w.sum(1,keepdims=True)+1e-9)
    return (w[:,:,None] * train_post[i]).sum(1)

post_te = knn_pull(Upos[tr], Psigned[tr], Upos[te], k=12)
acc_te  = accuracy_score(y[te], post_te.argmax(1))
print(f"[CNTC v2] Internal TEST acc (signed tuned): {acc_te:.3f}")

# -------------------------------
# 9) Diagnostics
# -------------------------------
n_comp, _ = connected_components(TopologyCapsule["Wpos"]>0, directed=False)
e6 = np.round(TopologyCapsule["evals_pos"][:6], 4)
cons_pos = (np.mean((i_pos>=0).astype(np.float32))*100.0)
cons_neg = (np.mean((i_neg_raw>=0).astype(np.float32))*100.0)
print(f"[Diag] components:{n_comp} | eigs[:6]={e6} | pos-kept≈{cons_pos:.1f}% | neg-kept≈{cons_neg:.1f}%")
print(f"Done in {time.time()-t0:.1f}s.")


Graphs built in 24.5s | pos edges:48,651 | neg edges:34,444 | components:1 | deg_pos mean:4.93
[CNTC v2] Unlabeled acc | Pd: 0.769 | Pd(signed λ=0.0, ridge=1e-05): 0.769 | seed-LL=0.0000
[CNTC v2] Internal TEST acc (signed tuned): 0.760
[Diag] components:1 | eigs[:6]=[-0.    0.    0.01  0.01  0.01  0.01] | pos-kept≈100.0% | neg-kept≈100.0%
Done in 291.6s.


In [ ]:
# ================= CIFAR-10 Shape Capsule (CPU-fast, no deep nets) =================
!pip -q install scikit-image scikit-learn torchvision --upgrade

import numpy as np, time, math, random
from skimage.feature import hog
from skimage.color import rgb2gray
from torchvision import datasets, transforms
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity

# --------- knobs (tune here) ---------
SEED = 7
N_POINTS = 4000           # try 6000 if you want, still CPU ok
PCA_DIM  = 96
KNN_K    = 12
USE_KFN  = True           # False = KNN-only capsule
KFN_K    = 6
RIDGE    = 1e-4
LABELED_PER_CLASS = 30    # few labels for Pd (shape->construct)
SPLIT_FRAC = 0.85

rng = np.random.default_rng(SEED)
random.seed(SEED)
np.random.seed(SEED)

def log(*a): print(*a, flush=True)

# --------- 1) Load a slice ----------
log("Loading CIFAR-10 …")
tf = transforms.Compose([transforms.ToTensor()])
ds = datasets.CIFAR10(root="./data", train=True, download=True, transform=tf)
idx = np.arange(len(ds)); rng.shuffle(idx); idx = idx[:N_POINTS]

X = []
y = []
t0 = time.time()
for i in idx:
    img_t, lab = ds[i]              # torch [3,32,32] in [0,1]
    img = img_t.numpy().transpose(1,2,0)  # (32,32,3)
    y.append(lab)
    # --------- 2) Fast CPU features: HOG + simple color stats ----------
    g = rgb2gray(img)                                # float64 ok
    # HOG params tuned for 32x32 speed & signal
    f_hog = hog(g, pixels_per_cell=(4,4), cells_per_block=(2,2),
                orientations=8, feature_vector=True, visualize=False, block_norm="L2-Hys")
    # tiny color stats (mean/std per channel)
    m = img.mean(axis=(0,1)); s = img.std(axis=(0,1))+1e-9
    feat = np.concatenate([f_hog.astype(np.float32), m.astype(np.float32), s.astype(np.float32)], 0)
    X.append(feat)
X = np.stack(X,0).astype(np.float32)
y = np.array(y, dtype=np.int64)
log(f"Features: {X.shape} built in {time.time()-t0:.1f}s")

# --------- 3) PCA space (compact geometry coords) ----------
scaler = StandardScaler(with_mean=True, with_std=True)
Xp = scaler.fit_transform(X)
pca = PCA(n_components=PCA_DIM, random_state=SEED)
Z = pca.fit_transform(Xp).astype(np.float32)

# --------- 4) Graphs: mutual kNN (+ optional KFN) ----------
def knn_graph(Z, k):
    nn = NearestNeighbors(n_neighbors=k+1, metric='cosine').fit(Z)
    d, i = nn.kneighbors(Z)       # d in [0,2] for cosine distance
    i = i[:,1:]; d = d[:,1:]
    S = np.clip(1.0 - d, 0.0, 1.0).astype(np.float32)  # similarity
    return i, S

def kfn_graph(Z, k, trim_q=0.02, t=0.0, s=0.35, mutual=True):
    # cosine similarity matrix (dense but N up to ~6k is fine on Colab CPU)
    sims = cosine_similarity(Z, Z).astype(np.float32)
    np.fill_diagonal(sims, np.inf)                 # never self
    order = np.argsort(sims, axis=1)               # ascending sim → farthest first
    k_eff = min(k + max(2, k//2), sims.shape[1]-1)
    cand_idx = order[:, :k_eff]
    cand_sim = np.take_along_axis(sims, cand_idx, axis=1)

    trim = max(1, int(math.ceil(trim_q * k_eff)))
    cand_idx = cand_idx[:, trim:trim+k]
    cand_sim = cand_sim[:, trim:trim+k]

    # soft repulsion weights via sigmoid
    R = 1.0 / (1.0 + np.exp((cand_sim - t) / s))
    R = np.maximum(R, 1e-6).astype(np.float32)

    if mutual:
        N = Z.shape[0]
        M = np.zeros((N,N), dtype=bool)
        rows = np.repeat(np.arange(N)[:,None], cand_idx.shape[1], 1)
        M[rows, cand_idx] = True
        keep = M[cand_idx, np.repeat(np.arange(N)[:,None], cand_idx.shape[1], 1)]
        # ensure at least one kept per row
        for r in range(keep.shape[0]):
            if not keep[r].any():
                keep[r, np.argmax(R[r])] = True
        # zero out non-mutual
        for r in range(R.shape[0]):
            mask = keep[r]
            if mask.sum() < R.shape[1]:
                need = R.shape[1] - mask.sum()
                top_extra = np.argsort(-R[r,~mask])[:need]
                idx_extra = np.flatnonzero(~mask)[top_extra]
                mask[idx_extra] = True
            R[r, ~mask] = 1e-6
    return cand_idx, R

def build_sym(indices, weights, N):
    W = np.zeros((N,N), dtype=np.float32)
    rows = np.repeat(np.arange(N)[:,None], indices.shape[1], 1)
    W[rows, indices] = weights
    return np.maximum(W, W.T)

def sym_normalize(W):
    d = W.sum(1) + 1e-8
    Dmh = np.diag(1.0/np.sqrt(d))
    return (Dmh @ W @ Dmh).astype(np.float32)

t0 = time.time()
i_pos, w_pos = knn_graph(Z, KNN_K)
Wpos = build_sym(i_pos, w_pos, N_POINTS)

if USE_KFN:
    i_neg, w_neg = kfn_graph(Z, KFN_K, trim_q=0.02, t=0.0, s=0.35, mutual=True)
    Wneg = build_sym(i_neg, w_neg, N_POINTS)
else:
    Wneg = np.zeros_like(Wpos)

Lpos = np.eye(N_POINTS, dtype=np.float32) - sym_normalize(Wpos)
Lnegm = np.eye(N_POINTS, dtype=np.float32) + sym_normalize(Wneg)
log(f"Graphs built in {time.time()-t0:.1f}s | pos edges: {(Wpos>0).sum()//2:,} | neg edges: {(Wneg>0).sum()//2:,}")

# --------- 5) Capsule-only label propagation (Pd) ----------
def few_labels_per_class(y, n_per_class, pool_idx):
    rng = np.random.default_rng(SEED)
    idxL=[]
    C=int(y.max())+1
    for c in range(C):
        cand = pool_idx[y[pool_idx]==c]
        if len(cand)==0: continue
        pick = cand if len(cand)<=n_per_class else rng.choice(cand, size=n_per_class, replace=False)
        idxL.extend(pick.tolist())
    idxL = np.array(sorted(set(idxL)), dtype=int)
    idxU = np.setdiff1d(pool_idx, idxL, assume_unique=False)
    return idxL, idxU

def signed_lp(y, Lpos, Lnegm, idxL, lam_neg=0.2, ridge=RIDGE):
    N = Lpos.shape[0]
    C = int(y.max())+1
    A = (Lpos + lam_neg*Lnegm + ridge*np.eye(N, dtype=np.float32)).astype(np.float32)
    Y = np.zeros((N,C), dtype=np.float32)
    Y[idxL, y[idxL]] = 1.0
    idxU = np.setdiff1d(np.arange(N), idxL, assume_unique=False)
    A_uu = A[np.ix_(idxU, idxU)]
    A_ul = A[np.ix_(idxU, idxL)]
    F_u  = np.linalg.solve(A_uu, -A_ul @ Y[idxL])
    F = np.zeros((N,C), dtype=np.float32); F[idxL]=Y[idxL]; F[idxU]=F_u
    P = np.clip(F,1e-9,None); P /= (P.sum(1,keepdims=True)+1e-9)
    return P

C = int(y.max())+1
split = int(SPLIT_FRAC * N_POINTS)
train_ids = np.arange(N_POINTS)[:split]
test_ids  = np.arange(N_POINTS)[split:]
idxL, idxU = few_labels_per_class(y, LABELED_PER_CLASS, train_ids)

Pd_knn   = signed_lp(y, Lpos, 0*Lnegm, idxL, lam_neg=0.0, ridge=RIDGE)
Pd_signed= signed_lp(y, Lpos,   Lnegm, idxL, lam_neg=0.2, ridge=RIDGE)

acc_knn   = accuracy_score(y[idxU], Pd_knn[idxU].argmax(1))
acc_signed= accuracy_score(y[idxU], Pd_signed[idxU].argmax(1))
log(f"[Capsule(HOG+PCA)] Unlabeled acc | KNN-only: {acc_knn:.3f} | KNN+KFN(signed): {acc_signed:.3f}")

# --------- 6) Internal TEST (capsule transfer) ----------
def knn_pull(Ztr, Ptr, Zte, k=KNN_K):
    nn = NearestNeighbors(n_neighbors=k, metric='cosine').fit(Ztr)
    d,i = nn.kneighbors(Zte)
    w = 1.0 - d; w /= (w.sum(1,keepdims=True)+1e-9)
    return (w[:,:,None] * Ptr[i]).sum(1)

post_test = knn_pull(Z[train_ids], Pd_signed[train_ids], Z[test_ids], k=KNN_K)
acc_test  = accuracy_score(y[test_ids], post_test.argmax(1))
log(f"[Capsule(HOG+PCA)] Internal TEST acc (signed): {acc_test:.3f}")

# --------- 7) Tiny diagnostics ----------
deg_pos = Wpos.sum(1).mean()
log(f"Done in {time.time()-t0:.1f}s | deg_pos mean: {deg_pos:.2f} | N={N_POINTS} | m={PCA_DIM}")


Loading CIFAR-10 …
Features: (4000, 1574) built in 14.6s
Graphs built in 9.6s | pos edges: 36,322 | neg edges: 19,882
[Capsule(HOG+PCA)] Unlabeled acc | KNN-only: 0.359 | KNN+KFN(signed): 0.352
[Capsule(HOG+PCA)] Internal TEST acc (signed): 0.367
Done in 13.3s | deg_pos mean: 7.27 | N=4000 | m=96


In [ ]:
# ===============================================
# CIFAR-10 Topology Capsule (HOG+color, PCA→KNN+KFN)
# - CPU-only, fast (~45–60s on Colab CPU @ N=4000)
# - No training loops, no pixels kept after capsule
# - Label inference via tuned signed label propagation
# ===============================================

!pip -q install torchvision scikit-image scikit-learn

import time, math, random, numpy as np
from collections import Counter

from skimage.feature import hog
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, log_loss
from sklearn.utils import check_random_state
from torchvision import datasets, transforms

from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components

# -------------------------------
# Config
# -------------------------------
SEED = 7
rng = np.random.default_rng(SEED)
random.seed(SEED)
np.random.seed(SEED)

N_POINTS   = 4000       # keep small for speed; try 8000 once you're happy
PCA_DIM    = 96
KNN_K      = 12
KFN_K      = 6
LABELED_PER_CLASS = 40  # seeds per class for capsule inference
LAM_GRID   = [0.0, 0.05, 0.1, 0.2]
RIDGE_GRID = [1e-5, 1e-4, 5e-4]

# -------------------------------
# Small utils
# -------------------------------
def normalize_rows(Z):
    n = np.linalg.norm(Z, axis=1, keepdims=True) + 1e-9
    return Z / n

def few_labels_per_class(y, n_per_class):
    idx = np.arange(len(y)); rng.shuffle(idx)
    bins = [[] for _ in range(int(y.max())+1)]
    for i in idx:
        c = int(y[i])
        if len(bins[c]) < n_per_class:
            bins[c].append(i)
        if all(len(b)==n_per_class for b in bins):
            break
    L = np.array([j for b in bins for j in b], dtype=int)
    U = np.setdiff1d(np.arange(len(y)), L, assume_unique=False)
    return L, U

def sym_normalize(W):
    d = W.sum(1) + 1e-8
    Dmh = np.diag(1.0/np.sqrt(d))
    return (Dmh @ W @ Dmh).astype(np.float32)

def build_sym(indices, weights, N):
    W = np.zeros((N,N), dtype=np.float32)
    rows = np.repeat(np.arange(N)[:,None], indices.shape[1], 1)
    W[rows, indices] = weights
    W = np.maximum(W, W.T)
    return W

def knn_pairs_cosine(Z, k):
    nbrs = NearestNeighbors(n_neighbors=k+1, metric='cosine').fit(Z)
    d, i = nbrs.kneighbors(Z)
    return i[:,1:], np.clip(1.0 - d[:,1:], 0.0, 1.0).astype(np.float32)

# ---- FIXED: farthest neighbors by cosine with correct normalization ----
def kfn_pairs_cosine(Z, k, trim_q=0.02, t=0.0, s=0.35):
    """
    Farthest-neighbor pairs by cosine dissimilarity on row-normalized Z.
    Returns indices (N x k) and soft repulsion weights (N x k).
    """
    Zhat = normalize_rows(Z.astype(np.float32))        # (N,d)
    sims = Zhat @ Zhat.T                               # cosine in [-1,1]
    np.fill_diagonal(sims, np.inf)                     # never select self as farthest

    # sort ascending by sim  → most dissimilar first
    order   = np.argsort(sims, axis=1)
    k_eff   = min(k + max(2, k//2), sims.shape[1]-1)   # oversample, then trim tails
    cand_ix = order[:, :k_eff]
    cand_sm = np.take_along_axis(sims, cand_ix, axis=1)

    # trim extreme outliers then keep top-k farthest
    trim = max(1, int(math.ceil(trim_q * k_eff)))
    cand_ix = cand_ix[:, trim:trim+k]
    cand_sm = cand_sm[:, trim:trim+k]

    # soft repulsion weights via a smooth sigmoid on similarity (lower sim → stronger repulsion)
    R = 1.0 / (1.0 + np.exp((cand_sm - t) / s))
    return cand_ix, np.maximum(R.astype(np.float32), 1e-6)

def laplacians(Wpos, Wneg):
    Spos = sym_normalize(Wpos)
    Sneg = sym_normalize(Wneg)
    Lpos = np.eye(Wpos.shape[0], dtype=np.float32) - Spos
    Lneg_minus = np.eye(Wneg.shape[0], dtype=np.float32) + Sneg
    return Lpos, Lneg_minus

def signed_lp(y, Lpos, Lnegm, idxL, lam_neg=0.1, ridge=1e-4):
    N = Lpos.shape[0]
    C = int(y.max())+1
    A = (Lpos + lam_neg*Lnegm + ridge*np.eye(N, dtype=np.float32)).astype(np.float32)

    Y = np.zeros((N, C), dtype=np.float32)
    Y[idxL, y[idxL]] = 1.0

    idxU = np.setdiff1d(np.arange(N), idxL, assume_unique=False)
    A_uu = A[np.ix_(idxU, idxU)]
    A_ul = A[np.ix_(idxU, idxL)]
    F_u  = np.linalg.solve(A_uu, -A_ul @ Y[idxL])
    F = np.zeros((N, C), dtype=np.float32); F[idxL]=Y[idxL]; F[idxU]=F_u
    P = np.clip(F, 1e-9, None); P /= (P.sum(1, keepdims=True)+1e-9)
    return P

def seed_loglik(P, y, idxL):
    Pclip = np.clip(P[idxL, y[idxL]], 1e-12, 1.0)
    return float(np.mean(np.log(Pclip)))

def signed_lp_tuned(y, Lpos, Lnegm, idxL, lam_grid, ridge_grid):
    best = (-np.inf, (None,None), None)
    for lam in lam_grid:
        for rg in ridge_grid:
            P_try = signed_lp(y, Lpos, Lnegm, idxL, lam_neg=lam, ridge=rg)
            ll = seed_loglik(P_try, y, idxL)
            if ll > best[0]:
                best = (ll, (lam, rg), P_try)
    return best[2], best[1], best[0]

# -------------------------------
# Feature extraction (HOG + color hist)
# -------------------------------
def hog_color_features(images):
    """
    images: (N, 32, 32, 3) float32 in [0,1]
    Returns: (N, D) features (HOG on gray + 16-bin color hist/channel)
    """
    N = images.shape[0]
    feats = []
    bins = 16
    for i in range(N):
        img = images[i]
        gray = (0.299*img[:,:,0] + 0.587*img[:,:,1] + 0.114*img[:,:,2]).astype(np.float32)
        # HOG (fast parameters for 32x32)
        h = hog(gray, orientations=9, pixels_per_cell=(8,8), cells_per_block=(2,2),
                block_norm='L2-Hys', feature_vector=True)
        # 16-bin per-channel hist (normalized)
        ch = []
        for c in range(3):
            hist, _ = np.histogram(img[:,:,c], bins=bins, range=(0,1), density=True)
            ch.append(hist.astype(np.float32))
        feat = np.concatenate([h.astype(np.float32)] + ch, axis=0)
        feats.append(feat)
    return np.stack(feats, 0)

# -------------------------------
# 1) Load CIFAR-10 and build features
# -------------------------------
t0 = time.time()
tf = transforms.Compose([transforms.ToTensor()])
train = datasets.CIFAR10(root='./data', train=True, download=True, transform=tf)

idx = np.arange(len(train))
rng.shuffle(idx); idx = idx[:N_POINTS]

X = []
y = []
for i in idx:
    img, lab = train[i]           # img: [3,32,32] tensor in [0,1]
    X.append(np.moveaxis(img.numpy(), 0, -1))  # (32,32,3)
    y.append(lab)
X = np.stack(X,0).astype(np.float32)
y = np.array(y, dtype=np.int64)

# HOG+color features + standardize + PCA
t_feat = time.time()
F = hog_color_features(X)                       # (N, D_hog+48)
scaler = StandardScaler().fit(F)
Fp = scaler.transform(F)
pca = PCA(n_components=PCA_DIM, random_state=SEED)
Z = pca.fit_transform(Fp).astype(np.float32)
print(f"Loading CIFAR-10 …\nFeatures: HOG+color {F.shape} in {time.time()-t_feat:.1f}s")

# -------------------------------
# 2) Build graphs (KNN+, KFN-) and Laplacians
# -------------------------------
def build_capsule_graphs(Z):
    i_pos, w_pos = knn_pairs_cosine(Z, KNN_K)
    i_neg, w_neg = kfn_pairs_cosine(Z, KFN_K, trim_q=0.02, t=0.0, s=0.35)
    Wpos = build_sym(i_pos, w_pos, Z.shape[0])
    Wneg = build_sym(i_neg, w_neg, Z.shape[0])
    Lpos, Lnegm = laplacians(Wpos, Wneg)
    return Wpos, Wneg, Lpos, Lnegm

t_g = time.time()
Wpos, Wneg, Lpos, Lnegm = build_capsule_graphs(Z)
# graph diagnostics
N = Z.shape[0]
G = csr_matrix((Wpos>0).astype(np.int8))
ncomp, labels = connected_components(G, directed=False)
deg_mean = Wpos.sum(1).mean()
print(f"Graphs built in {time.time()-t_g:.1f}s | pos edges: {int((Wpos>0).sum())} | neg edges: {int((Wneg>0).sum())} | components: {ncomp} | deg_pos mean: {deg_mean:.2f}")

# -------------------------------
# 3) Seeds and label propagation (tuned)
# -------------------------------
idxL, idxU = few_labels_per_class(y, LABELED_PER_CLASS)
P_knn, _, _ = signed_lp_tuned(y, Lpos, 0.0*Lnegm, idxL, lam_grid=[0.0], ridge_grid=[1e-5])  # KNN-only baseline
acc_knn = accuracy_score(y[idxU], P_knn[idxU].argmax(1))

P_signed, (lam_opt, rg_opt), ll = signed_lp_tuned(y, Lpos, Lnegm, idxL, LAM_GRID, RIDGE_GRID)
acc_signed = accuracy_score(y[idxU], P_signed[idxU].argmax(1))
print(f"[Capsule(HOG+PCA)] Unlabeled acc | KNN-only: {acc_knn:.3f} | KNN+KFN(signed): {acc_signed:.3f}")

# -------------------------------
# 4) Internal TEST accuracy (held-out slice, capsule-only transfer)
# -------------------------------
split = int(0.85 * N)
test_ids = np.arange(N)[split:]
acc_test = accuracy_score(y[test_ids], P_signed[test_ids].argmax(1))
print(f"[Capsule(HOG+PCA)] Internal TEST acc (signed): {acc_test:.3f}")
print(f"Done in {time.time()-t0:.1f}s | N={N} | m={PCA_DIM} | (lam={lam_opt}, ridge={rg_opt}, seed-LL={ll:.4f})")


Loading CIFAR-10 …
Features: HOG+color (4000, 372) in 3.5s
Graphs built in 9.8s | pos edges: 72094 | neg edges: 40462 | components: 1 | deg_pos mean: 7.81
[Capsule(HOG+PCA)] Unlabeled acc | KNN-only: 0.373 | KNN+KFN(signed): 0.373
[Capsule(HOG+PCA)] Internal TEST acc (signed): 0.412
Done in 36.8s | N=4000 | m=96 | (lam=0.0, ridge=1e-05, seed-LL=0.0000)


In [ ]:
# ===============================================
# Cat vs Dog — Multi-View Topology Capsule (fast CPU)
# - Views: silhouette (sil), texture (tex), structure (str), diffused (dif)
# - Graphs: KNN(+) + KFN(-) per-view, signed LP posteriors Pd_view
# - Per-view temperature calibration on seeds
# - Stability weights via light perturb-and-resolve
# - Geometric-mean ensemble (no logits, no NaNs)
# ===============================================

!pip -q install torchvision scikit-image

import numpy as np, math, time, random
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from torchvision import datasets, transforms
from skimage import color
from skimage.feature import hog, local_binary_pattern
from skimage.filters import threshold_otsu
from skimage.transform import resize
from skimage.filters import gabor
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# -------------------------------
# Repro & knobs
# -------------------------------
SEED             = 7
rng              = np.random.default_rng(SEED)
random.seed(SEED)
np.random.seed(SEED)

N_POINTS         = 4000         # subset for speed
LABELED_PER_CLS  = 60           # seeds per class (binary here → total 120)
KPOS             = 12           # positive K (KNN)
KNEG             = 8            # negative K (KFN)
PCA_DIM_SIL      = 64           # compact distance space per-view
PCA_DIM_STR      = 96
PCA_DIM_TEX      = 48
PERTURB_REPEATS  = 3            # stability weighting budget
PERTURB_NOISE    = 0.08         # multiplicative noise on edges for stability
DIFF_T           = 1.2          # heat smoothing for "diff" view
RIDGE_GRID       = [1e-5, 5e-5, 1e-4]
LAMNEG_GRID      = [0.0, 0.1, 0.2]

# -------------------------------
# Small utils
# -------------------------------
def split_train_test(N, test_frac=0.15, seed=SEED):
    idx = np.arange(N)
    rng = np.random.default_rng(seed)
    rng.shuffle(idx)
    s = int(round((1.0 - test_frac) * N))
    return idx[:s], idx[s:]

def few_labels_per_class(y, n_per_class):
    idx = np.arange(len(y)); rng.shuffle(idx)
    labels = np.unique(y)
    take = []
    cnt = {int(c): 0 for c in labels}
    for i in idx:
        c = int(y[i])
        if cnt[c] < n_per_class:
            cnt[c] += 1
            take.append(i)
        if all(cnt[int(c)] >= n_per_class for c in labels):
            break
    L = np.array(take, dtype=int)
    U = np.setdiff1d(np.arange(len(y)), L, assume_unique=False)
    return L, U

def normalize_rows(A):
    n = np.linalg.norm(A, axis=1, keepdims=True) + 1e-9
    return A / n

# -------------------------------
# 1) Load CIFAR-10 cats/dogs subset (labels: cat=0, dog=1)
# -------------------------------
t0 = time.time()
tf = transforms.Compose([transforms.ToTensor()])
ds = datasets.CIFAR10(root="./data", train=True, download=True, transform=tf)

# cat=3, dog=5 in CIFAR-10. Relabel to {0,1}
idx_all = [i for i in range(len(ds)) if ds.targets[i] in (3,5)]
rng.shuffle(idx_all)
idx_all = idx_all[:N_POINTS]

Xrgb = []
y = []
for i in idx_all:
    img, lab = ds[i]                 # img: [3,32,32] float in [0,1]
    Xrgb.append((img.numpy()*255.0).astype(np.uint8))   # uint8 for LBP stability
    y.append(0 if lab==3 else 1)     # {cat,dog} -> {0,1}
Xrgb = np.stack(Xrgb, 0)             # (N, 3, 32, 32) uint8
y = np.array(y, dtype=np.int64)
print(f"Loaded cat/dog subset: N={len(y)} (cat={(y==0).sum()}, dog={(y==1).sum()})")

# -------------------------------
# 2) View feature builders (fast)
# -------------------------------
def silhouette_view(img_u8):
    # grayscale float32
    gray_f = color.rgb2gray(np.moveaxis(img_u8,0,2).astype(np.float32)/255.0)
    # coarse silhouette via Otsu threshold
    t = threshold_otsu(gray_f)
    mask = (gray_f < t).astype(np.float32)
    # normalize area & simple radial bins
    mask_r = resize(mask, (24,24), anti_aliasing=True).astype(np.float32)
    # 1) downsampled mask
    v1 = resize(mask, (12,12), anti_aliasing=True).reshape(-1).astype(np.float32)
    # 2) row/col sums (shape profile)
    v2 = np.concatenate([mask_r.sum(0), mask_r.sum(1)], 0).astype(np.float32)
    # 3) coarse contour via Sobel energy
    from skimage.filters import sobel
    edge = sobel(mask_r)
    v3 = edge.reshape(-1)
    v = np.concatenate([v1, v2, v3], 0)
    return v

def texture_view(img_u8):
    # LBP prefers integer domain; use uint8 on [0,255]
    gray_u8 = color.rgb2gray(np.moveaxis(img_u8,0,2)).astype(np.float32)
    gray_u8 = np.clip(gray_u8*255.0, 0, 255).astype(np.uint8)

    def lbp_hist(gray_u8, P, R):
        lbp = local_binary_pattern(gray_u8, P=P, R=R, method='uniform')
        n_bins = P + 2
        hist, _ = np.histogram(lbp.ravel(), bins=n_bins, range=(0, n_bins), density=True)
        return hist.astype(np.float32)

    h1 = lbp_hist(gray_u8, P=8,  R=1)   # 10
    h2 = lbp_hist(gray_u8, P=16, R=2)   # 18
    h3 = lbp_hist(gray_u8, P=8,  R=2)   # 10  -> total 38 bins

    # Gabor energies (on float grayscale)
    gray_f = color.rgb2gray(np.moveaxis(img_u8,0,2).astype(np.float32)/255.0)
    real, imag = gabor(gray_f, frequency=0.25, theta=0.0);  e0 = np.mean(real**2 + imag**2)
    real, imag = gabor(gray_f, frequency=0.25, theta=np.pi/2); e1 = np.mean(real**2 + imag**2)
    gab = np.array([e0, e1], dtype=np.float32)              # +2 -> 40 dims
    return np.concatenate([h1,h2,h3,gab], 0)

def structure_view(img_u8):
    # HOG on luminance — light settings to keep dim ~150
    gray_f = color.rgb2gray(np.moveaxis(img_u8,0,2).astype(np.float32)/255.0)
    feat = hog(gray_f, orientations=8, pixels_per_cell=(8,8), cells_per_block=(2,2),
               block_norm='L2-Hys', feature_vector=True)
    return feat.astype(np.float32)

# Build raw features
t1 = time.time()
V_sil = np.stack([silhouette_view(x) for x in Xrgb], 0)
V_tex = np.stack([texture_view(x)    for x in Xrgb], 0)
V_str = np.stack([structure_view(x)  for x in Xrgb], 0)
print(f"Built raw view features: sil={V_sil.shape}, tex={V_tex.shape}, str={V_str.shape} in {time.time()-t1:.1f}s")

# Compact metric spaces (per-view PCA over standardized features)
def compact_embed(V, dim):
    Vz = StandardScaler().fit_transform(V)
    if dim >= Vz.shape[1]:
        return Vz.astype(np.float32)
    pca = PCA(n_components=dim, random_state=SEED)
    return pca.fit_transform(Vz).astype(np.float32)

Z_sil = compact_embed(V_sil, PCA_DIM_SIL)
Z_tex = compact_embed(V_tex, PCA_DIM_TEX)
Z_str = compact_embed(V_str, PCA_DIM_STR)

# -------------------------------
# 3) Graphs: KNN(+) + KFN(-) (cosine), symmetrized
# -------------------------------
def knn_pairs_cosine(Z, k):
    nbrs = NearestNeighbors(n_neighbors=k+1, metric='cosine').fit(Z)
    d, i = nbrs.kneighbors(Z)
    i = i[:,1:]
    S = np.clip(1.0 - d[:,1:], 0.0, 1.0).astype(np.float32)
    return i, S

def kfn_pairs_cosine(Z, k, trim_q=0.02, t=0.0, s=0.35):
    # cosine similarity in [-1,1]; choose smallest sims (farthest)
    ZN = normalize_rows(Z)
    sims = (ZN @ ZN.T).astype(np.float32)
    np.fill_diagonal(sims, np.inf)
    order = np.argsort(sims, axis=1)         # ascending -> farthest first
    k_eff = min(k + max(2,k//2), Z.shape[0]-1)
    cand_idx = order[:, :k_eff]
    cand_sim = np.take_along_axis(sims, cand_idx, axis=1)
    # trim extremes
    trim = max(1, int(math.ceil(trim_q * k_eff)))
    cand_idx = cand_idx[:, trim:trim+k]
    cand_sim = cand_sim[:, trim:trim+k]
    # soft repulsion
    R = 1.0 / (1.0 + np.exp((cand_sim - t) / s))
    return cand_idx.astype(np.int32), np.maximum(R, 1e-6).astype(np.float32)

def build_sym(indices, weights, N):
    W = np.zeros((N,N), dtype=np.float32)
    rows = np.repeat(np.arange(N)[:,None], indices.shape[1], 1)
    W[rows, indices] = weights
    return np.maximum(W, W.T)

def sym_norm(W):
    d = W.sum(1) + 1e-8
    Dmh = np.diag(1.0/np.sqrt(d))
    return (Dmh @ W @ Dmh).astype(np.float32)

def capsule_graphs(Z):
    i_pos, w_pos = knn_pairs_cosine(Z, KPOS)
    i_neg, w_neg = kfn_pairs_cosine(Z, KNEG, trim_q=0.02, t=0.0, s=0.35)
    Wpos = build_sym(i_pos, w_pos, Z.shape[0])
    Wneg = build_sym(i_neg, w_neg, Z.shape[0])
    Spos = sym_norm(Wpos)
    Sneg = sym_norm(Wneg)
    Lpos = (np.eye(Z.shape[0], dtype=np.float32) - Spos).astype(np.float32)
    Lnegm= (np.eye(Z.shape[0], dtype=np.float32) + Sneg).astype(np.float32)
    return Wpos, Wneg, Lpos, Lnegm

t2 = time.time()
Wpos_sil, Wneg_sil, Lpos_sil, Lnegm_sil = capsule_graphs(Z_sil)
Wpos_tex, Wneg_tex, Lpos_tex, Lnegm_tex = capsule_graphs(Z_tex)
Wpos_str, Wneg_str, Lpos_str, Lnegm_str = capsule_graphs(Z_str)
print(f"Graphs built in {time.time()-t2:.1f}s | pos edges: {int(Wpos_sil.sum()>0)+int(Wpos_tex.sum()>0)+int(Wpos_str.sum()>0)} (per-view matrices built)")

# -------------------------------
# 4) Seeds & signed LP (tune on seeds by log-likelihood)
# -------------------------------
idxL, idxU = few_labels_per_class(y, LABELED_PER_CLS)

def signed_lp_tuned(y, Lpos, Lnegm, idxL, lam_grid=LAMNEG_GRID, ridge_grid=RIDGE_GRID):
    N = Lpos.shape[0]
    C = int(y.max())+1
    YL = np.zeros((len(idxL), C), dtype=np.float32); YL[np.arange(len(idxL)), y[idxL]] = 1.0
    best = (-np.inf, None, None, None)  # (ll, lam, ridge, P)

    for lam in lam_grid:
        A = (Lpos + lam*Lnegm).astype(np.float32)
        for rg in ridge_grid:
            A_ = A + rg*np.eye(N, dtype=np.float32)
            idxU_local = np.setdiff1d(np.arange(N), idxL, assume_unique=False)
            A_uu = A_[np.ix_(idxU_local, idxU_local)]
            A_ul = A_[np.ix_(idxU_local, idxL)]
            Fu  = np.linalg.solve(A_uu, -A_ul @ YL)
            F   = np.zeros((N,C), dtype=np.float32); F[idxL]=YL; F[idxU_local]=Fu
            P   = np.clip(F, 1e-6, None); P /= (P.sum(1, keepdims=True)+1e-9)
            # seed LL
            ll = float(np.mean(np.log(np.clip(P[idxL, y[idxL]], 1e-9, 1.0))))
            if ll > best[0]:
                best = (ll, lam, rg, P)
    return best[3], (best[1], best[2]), best[0]

# per-view posteriors
P_sil, (lam_sil, rg_sil), ll_sil = signed_lp_tuned(y, Lpos_sil, Lnegm_sil, idxL)
P_tex, (lam_tex, rg_tex), ll_tex = signed_lp_tuned(y, Lpos_tex, Lnegm_tex, idxL)
P_str, (lam_str, rg_str), ll_str = signed_lp_tuned(y, Lpos_str, Lnegm_str, idxL)

# An extra "diffused" view: heat-smooth P_str over its graph
def heat_smooth(P, Lpos, t=DIFF_T):
    # Use (I + t*Lpos)^{-1} P  (one-step Tikhonov heat approximation)
    N = Lpos.shape[0]
    A = (np.eye(N, dtype=np.float32) + t*Lpos).astype(np.float32)
    return np.linalg.solve(A, P).astype(np.float32)

P_diff = heat_smooth(P_str, Lpos_str, t=DIFF_T)
for P in (P_sil, P_tex, P_str, P_diff):
    P[:] = np.clip(P, 1e-6, 1.0); P[:] /= (P.sum(1, keepdims=True)+1e-9)

# quick per-view reference
acc_sil = accuracy_score(y[idxU], P_sil[idxU].argmax(1))
acc_tex = accuracy_score(y[idxU], P_tex[idxU].argmax(1))
acc_str = accuracy_score(y[idxU], P_str[idxU].argmax(1))
acc_dif = accuracy_score(y[idxU], P_diff[idxU].argmax(1))
print(f"[Per-view] Unlabeled acc — sil:{acc_sil:.3f} tex:{acc_tex:.3f} str:{acc_str:.3f} dif:{acc_dif:.3f}")

# -------------------------------
# 5) Per-view temperature calibration (on seeds)
# -------------------------------
def calibrate_T(P, idx, y_true, grid=(0.6, 0.8, 1.0, 1.2, 1.6)):
    P = np.clip(P, 1e-6, 1-1e-6)
    logP = np.log(P)
    bestT, bestLL = 1.0, -np.inf
    for T in grid:
        Q = np.exp(logP[idx] / T); Q /= (Q.sum(1, keepdims=True) + 1e-9)
        ll = float(np.mean(np.log(np.clip(Q[np.arange(len(idx)), y_true[idx]], 1e-9, 1.0))))
        if ll > bestLL:
            bestLL, bestT = ll, T
    return bestT, bestLL

def apply_T(P, T):
    P = np.clip(P, 1e-6, 1-1e-6)
    Q = np.exp(np.log(P)/T); Q /= (Q.sum(1, keepdims=True)+1e-9)
    return Q

Ts = []
seedLLs = []
views_raw = [("sil", P_sil), ("tex", P_tex), ("str", P_str), ("dif", P_diff)]
views_cal = {}
for name, Pv in views_raw:
    T, ll = calibrate_T(Pv, idxL, y)
    Ts.append(T); seedLLs.append(ll)
    views_cal[name] = apply_T(Pv, T)

# -------------------------------
# 6) Stability weighting (light perturb-and-resolve)
# -------------------------------
def stability_weight(Lpos, Lnegm, P_ref, idxL, repeats=PERTURB_REPEATS, noise=PERTURB_NOISE):
    # perturb W via Lpos' = I - (D^{-1/2}(W*(1+eps))D^{-1/2})   approx through Lpos scaling
    # we approximate by scaling Laplacian: L' = (I + eps) L  (cheap proxy)
    N = Lpos.shape[0]
    C = P_ref.shape[1]
    YL = np.zeros((len(idxL), C), dtype=np.float32); YL[np.arange(len(idxL)), y[idxL]] = 1.0
    idxU_local = np.setdiff1d(np.arange(N), idxL, assume_unique=False)

    # choose a fixed (lam, ridge) that worked on seeds for this view by re-estimating quickly
    best = signed_lp_tuned(y, Lpos, Lnegm, idxL)[1]
    lam, rg = best
    A = (Lpos + lam*Lnegm + rg*np.eye(N, dtype=np.float32)).astype(np.float32)
    A_uu = A[np.ix_(idxU_local, idxU_local)]
    A_ul = A[np.ix_(idxU_local, idxL)]

    def solve(A_uu, A_ul):
        Fu  = np.linalg.solve(A_uu, -A_ul @ YL)
        F   = np.zeros((N, C), dtype=np.float32); F[idxL]=YL; F[idxU_local]=Fu
        P   = np.clip(F, 1e-6, None); P /= (P.sum(1, keepdims=True)+1e-9)
        return P

    Pref = solve(A_uu, A_ul)
    # perturb a few times
    deltas = []
    for _ in range(repeats):
        eps = (1.0 + noise * rng.standard_normal(size=(1,), dtype=np.float32))[0]
        A_uu_p = A_uu * eps
        A_ul_p = A_ul * eps
        Pp = solve(A_uu_p, A_ul_p)
        # mean cosine similarity between Pref and Pp
        u = normalize_rows(Pref)
        v = normalize_rows(Pp)
        cos = (u*v).sum(1).mean()
        deltas.append(float(cos))
    # stability score: higher is better
    return max(0.0, min(1.0, float(np.mean(deltas))))

w_sil = stability_weight(Lpos_sil, Lnegm_sil, views_cal["sil"], idxL)
w_tex = stability_weight(Lpos_tex, Lnegm_tex, views_cal["tex"], idxL)
w_str = stability_weight(Lpos_str, Lnegm_str, views_cal["str"], idxL)
w_dif = stability_weight(Lpos_str, Lnegm_str, views_cal["dif"], idxL)
stab = np.array([w_sil, w_tex, w_str, w_dif], dtype=np.float32)

# evidence from seeds (calibrated)
seed_e = np.exp(np.array(seedLLs, dtype=np.float32))
raw_w  = seed_e * stab
w_all  = raw_w / (raw_w.sum() + 1e-12)
print("Per-view T:", [round(t,2) for t in Ts])
print("Seed LL:", [round(ll,4) for ll in seedLLs])
print("Stability:", [round(float(x),3) for x in stab])
print("Blend weights:", [round(float(x),3) for x in w_all])

# -------------------------------
# 7) Geometric-mean ensemble (numerically safe)
# -------------------------------
P_sil_c = np.clip(views_cal["sil"], 1e-6, 1-1e-6)
P_tex_c = np.clip(views_cal["tex"], 1e-6, 1-1e-6)
P_str_c = np.clip(views_cal["str"], 1e-6, 1-1e-6)
P_diff_c= np.clip(views_cal["dif"], 1e-6, 1-1e-6)

Lsum = (w_all[0]*np.log(P_sil_c) +
        w_all[1]*np.log(P_tex_c) +
        w_all[2]*np.log(P_str_c) +
        w_all[3]*np.log(P_diff_c))
P_ens = np.exp(Lsum); P_ens /= (P_ens.sum(1, keepdims=True) + 1e-9)

# -------------------------------
# 8) Metrics
# -------------------------------
acc_u_ens = accuracy_score(y[idxU], P_ens[idxU].argmax(1))
tr_ids, te_ids = split_train_test(len(y), test_frac=0.15, seed=SEED)
acc_t_ens = accuracy_score(y[te_ids], P_ens[te_ids].argmax(1))

print(f"[ENSEMBLE†] Unlabeled acc: {acc_u_ens:.3f} | Internal TEST acc: {acc_t_ens:.3f}")

# single-view references after calibration
acc_t_str = accuracy_score(y[te_ids], P_str_c[te_ids].argmax(1))
print(f"[Structure-only (calib)] Internal TEST acc: {acc_t_str:.3f}")

print(f"Done in {time.time()-t0:.1f}s.")

Loaded cat/dog subset: N=4000 (cat=1998, dog=2002)
Built raw view features: sil=(4000, 768), tex=(4000, 40), str=(4000, 288) in 26.5s
Graphs built in 28.1s | pos edges: 3 (per-view matrices built)
[Per-view] Unlabeled acc — sil:0.530 tex:0.542 str:0.584 dif:0.585
Per-view T: [0.6, 0.6, 0.6, 0.6]
Seed LL: [0.0, 0.0, 0.0, -0.1534]
Stability: [1.0, 1.0, 1.0, 1.0]
Blend weights: [0.259, 0.259, 0.259, 0.222]
[ENSEMBLE†] Unlabeled acc: 0.587 | Internal TEST acc: 0.608
[Structure-only (calib)] Internal TEST acc: 0.580
Done in 206.8s.
